In [1]:
# Import statements
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from sklearn.metrics.pairwise import cosine_similarity

import re
import os
from dotenv import load_dotenv
import numpy as np

c:\Users\yoush\NexiAI\myenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# load_dotenv
load_dotenv()

True

In [3]:
# Initialize Groq LLM
llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"), # api_key required
    model="llama-3.1-8b-instant",
    temperature=1.2,
    max_tokens=500 
)

In [4]:
# Embeddings Model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

C:\Users\yoush\AppData\Local\Temp\ipykernel_24684\1112972915.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [5]:
# Interview LangChain prompt format
question_prompt = ChatPromptTemplate.from_template("""
You are an expert interviewer. Generate {num_questions} unique, clear, field-specific questions.

Field: {field}
Tone: {tone}
                                                   
Return format:
1. Question 1
2. Question 2
3. Question 3                                                                                           
""")

In [6]:
# New Chain using LCEL
question_chain = question_prompt | llm | StrOutputParser()

In [7]:
# Ideal Answer Generation Prompt
ideal_answer_prompt = ChatPromptTemplate.from_template("""
You are an expert in {field}.
Provide a concise, high-quality IDEAL answer for the interview question below.
Question: {question}                                                     
Return only the answer.                                

""")

In [8]:
ideal_answer_chain = ideal_answer_prompt | llm | StrOutputParser()

In [9]:
# 🔥 NEW: Evaluation Prompt
evaluation_prompt = ChatPromptTemplate.from_template("""
You are an expert interviewer and evaluator.

Evaluate the candidate's interview based on the following questions and answers.

Question: {question}
Ideal Answer: {ideal_answer}
User Answer: {user_answer}
Similarity Score: {similarity_score}% 

                                                                               
Write evaluation in this:
                                                     
format Score (0-10): X
Feedback:
- point 1
- point 2

Suggestions for Improvement:
- point 1
                                                                                                       
Final Verdict (Good / Average / Poor):
""")

In [10]:
# New evaluation_chain using LCEL
evaluation_chain = evaluation_prompt | llm | StrOutputParser()

In [11]:
def start_interview(field: str, tone: str, num_questions: int = 5):

    # Step 1: Generate interview questions
    questions_text = question_chain.invoke({
        "field": field,
        "tone": tone,
        "num_questions": num_questions
    })

    questions = [q.strip() for q in questions_text.split("\n") if q.strip() and q.lstrip()[0].isdigit()]

    print(f"\n🎤 Starting {field} interview in a {tone} tone...\n")

    # FAISS index for ideal answers
    faiss_store = None
    ideal_answer_texts = []

    final_report = []
    total_similarity = 0
    total_score = 0

    for i, q in enumerate(questions, 1):

        print(f"Q{i}: {q}")
        user_answer = input("💭 Your answer: ").strip()

        if user_answer.lower() in ["exit", "quit"]:
            print("\n🛑 Interview ended by user.")
            break

        # Step 2: Get ideal answer for this question
        ideal_answer = ideal_answer_chain.invoke({
            "field": field,
            "question": q
        })

        ideal_answer_texts.append(ideal_answer)

        # Step 3: Add ideal answer to FAISS store
        if faiss_store is None:
            faiss_store = FAISS.from_texts([ideal_answer], embeddings)
        else:
            faiss_store.add_texts([ideal_answer])

        # Step 4: Use FAISS to get similarity
        result = faiss_store.similarity_search_with_score(user_answer, k=1)
        best_match, similarity_score = result[0]

        # FAISS similarity score is distance → convert to similarity %
        similarity_percent = round((1 - similarity_score) * 100, 2)

        total_similarity += similarity_percent

        # Step 5: Evaluate
        evaluation = evaluation_chain.invoke({
            "question": q,
            "ideal_answer": ideal_answer,
            "user_answer": user_answer,
            "similarity_score": similarity_percent
        })

        final_report.append({
            "question": q,
            "similarity": similarity_percent,
            "evaluation": evaluation
        })

        score_match = re.search(r"Score \(0-10\):\s*(\d+)", evaluation)
        if score_match:
            total_score += int(score_match.group(1))

    # -------------------------
    # FINAL SUMMARY
    # -------------------------

    if len(final_report) == 0:
        print("\n⚠ No answers provided — no summary generated.\n")
        return final_report

    print("\n\n🎯 FINAL INTERVIEW SUMMARY")
    print("=======================================\n")

    avg_similarity = round(total_similarity / len(final_report), 2)
    avg_score = round(total_score / len(final_report), 2)

    print(f"⭐ Average Similarity Score: {avg_similarity}%")
    print(f"🏆 Average Interview Score: {avg_score}/10")

    if avg_score >= 8:
        verdict = "Excellent"
    elif avg_score >= 6:
        verdict = "Good"
    elif avg_score >= 4:
        verdict = "Average"
    else:
        verdict = "Poor"

    print(f"\n📌 Final Verdict: {verdict}")
    print("\n---------------------------------------")
    print("📘 Detailed Per-Question Evaluation\n")

    for i, item in enumerate(final_report, 1):
        print(f"Q{i}: {item['question']}")
        print(f"Similarity: {item['similarity']}%")
        print(f"Evaluation:\n{item['evaluation']}")
        print("---------------------------------------")

    print("\n🎉 Interview Completed Successfully!")
    return final_report

In [12]:
# Run to start the interview
if __name__ == "__main__":
    start_interview(field="Data Science", tone="friendly",
num_questions=5)


🎤 Starting Data Science interview in a friendly tone...

Q1: 1. **Tell us about a time when you had to navigate a messy dataset and develop a creative solution to ensure accurate insights: what steps did you take and what skills did you use to overcome the challenge?**

🛑 Interview ended by user.

⚠ No answers provided — no summary generated.

